In [14]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
sns.set()

import networkx as nx

In [15]:
subjects = ["soggetto_1","soggetto_2","soggetto_3","soggetto_6","soggetto_7","soggetto_8"]

In [16]:
df_weights_dict = {}
for subject in subjects:
    df_weights_dict[subject] = pd.read_csv("Datasets/{}_weights_o.csv".format(subject))
    df_weights_dict[subject].columns = ["TrRi","RiTr","Plu","Ttra","Ttrb","Tr","MeTr","MoTr","RiRi","MoFo","Mmof","MoRi","MeRi","Ri","Comp"]
    df_weights_dict[subject].index = df_weights_dict[subject].columns
    df_weights_dict[subject] = df_weights_dict[subject].T

In [17]:
df_weights_dict["soggetto_1"]

,TrRi,RiTr,Plu,Ttra,Ttrb,Tr,MeTr,MoTr,RiRi,MoFo,Mmof,MoRi,MeRi,Ri,Comp
TrRi,-0.770991,-0.247862,0.296357,0.183060,0.184627,-0.101070,0.283836,-0.350846,-0.124972,0.235450,0.137425,-0.531965,0.264298,0.166439,-0.109050
RiTr,-0.751945,-0.213791,0.143637,0.483585,0.094576,0.339746,-0.172645,-0.118040,-0.444297,0.209074,0.092889,-0.012311,0.233756,0.050772,-0.083759
Plu,-0.003196,-0.263307,-0.274596,0.501303,0.255244,0.104130,0.084332,0.095562,-0.265006,0.323443,0.318363,-0.268569,-0.447416,0.243699,-0.026358
Ttra,-0.475967,-0.452790,0.401257,0.105587,0.079634,0.175691,-0.112213,0.115430,-0.028899,0.554424,0.219523,0.330389,0.093097,0.464091,0.069246
Ttrb,-0.288003,-0.036305,-0.513654,-0.269760,0.433292,0.178291,-0.095082,-0.201036,0.315952,0.071989,0.547407,-0.077660,0.216503,-0.664574,0.217016
Tr,-0.319896,-0.265057,-0.032019,0.243317,0.108078,0.192165,-0.118429,0.017604,-0.310293,0.374774,0.242819,-0.339650,-0.126303,-0.043444,0.013585
MeTr,-0.237846,-0.040311,-0.606530,0.465550,0.718556,-0.148726,0.377106,0.278993,-0.491082,0.079382,-0.216052,0.582619,0.145657,-0.663608,0.802352
MoTr,-0.667909,-0.216138,0.466011,0.118819,0.207032,0.298971,-0.212882,-0.181154,-0.395986,0.243083,0.133172,-0.161037,-0.140548,0.324216,-0.157806
RiRi,-0.311236,0.195006,0.234262,0.741436,0.218471,0.257728,0.259866,-0.201408,-0.170462,0.017236,0.197097,0.152464,-0.464821,0.192691,-0.134792
MoFo,-0.313909,-0.285269,0.095813,0.158044,0.145419,0.369778,-0.511182,0.201035,-0.281086,0.427540,0.252716,-0.123318,-0.092667,0.153888,0.135851


In [18]:
# for every weight matrix, create a directed graph
# create one node for each column
# create an edge for each non-zero value with the weight as the edge weight given by the matrix

G_dict = {}
for subject in subjects:
    # add the nodes
    G_dict[subject] = nx.DiGraph()
    G_dict[subject].add_nodes_from(df_weights_dict[subject].columns)
    # add the edges with the weights
    for i in range(len(df_weights_dict[subject].columns)):
        for j in range(len(df_weights_dict[subject].columns)):
            if df_weights_dict[subject].iloc[i,j] != 0:
                G_dict[subject].add_edge(df_weights_dict[subject].columns[i],df_weights_dict[subject].columns[j],weight=df_weights_dict[subject].iloc[i,j])

In [19]:
# for each graph, compute the PageRank and organize the results in a dataframe
# the dataframe will have the following columns:
# - node: the name of the node
# - page_rank: the PageRank value of the node
# - subject: the subject to which the node belongs

df_page_rank_dict = {}
for subject in subjects:
    df_page_rank_dict[subject] = pd.DataFrame(list(nx.pagerank( G_dict[subject], max_iter=1000).items()),columns=["node","page_rank"])
    df_page_rank_dict[subject]["subject"] = subject

df_page_rank_dict["soggetto_1"]

/home/davide/Desktop/LAVORO/SEMEION/neural-network-with-fuzzified-inverter/env/lib/python3.10/site-packages/numpy/core/_methods.py:49: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


PowerIterationFailedConvergence: (PowerIterationFailedConvergence(...), 'power iteration failed to converge within 1000 iterations')